# Some fun with &pi; in Julia

## &pi; in Julia
*([Simon Byrne](https://github.com/simonbyrne))*

Like most technical languages, Julia provides a variable constant for &pi;. However Julia's handling is a bit special.

In [1]:
pi

π = 3.1415926535897...

It can also be accessed via the unicode symbol (you can get it at the REPL or in a notebook via the TeX completion `\pi` followed by a tab)

In [2]:
π

π = 3.1415926535897...

You'll notice that it doesn't print like an ordinary floating point number: that's because it isn't one.

In [3]:
typeof(pi)

Irrational{:π}

&pi; and a few other irrational constants are instead stored as special `Irrational` values, rather than being rounded to `Float64`. These act like ordinary numeric values, except that they can are converted automatically to any floating point type without any intermediate rounding:

In [4]:
1 + pi # integers are promoted to Float64 by default

4.141592653589793

In [5]:
Float32(1) + pi # Float32

4.141593f0

This is particularly useful for use with arbitrary-precision `BigFloat`s, as &pi; can be evaluated to full precision (rather than be truncated to `Float64` and converted back).

In [6]:
BigFloat(1) + pi # 256 bits by default

4.141592653589793238462643383279502884197169399375105820974944592307816406286198

If &pi; were stored as a `Float64`, we would instead get

In [7]:
BigFloat(1) + Float64(pi)

4.141592653589793115997963468544185161590576171875000000000000000000000000000000

In fact `BigFloat` (which uses the [MPFR](http://www.mpfr.org) library) will compute &pi; on demand to the current precision, which is set via `setprecision`. This provides an easy way to get its digits: 

In [8]:
# to 1024 bits
setprecision(BigFloat, 1024) do 
    BigFloat(pi)
end

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724586997

The last few digits may be incorrect due to the conversion from the internal binary format of `BigFloat` to the decimal representation used for printing.

Another neat property of `Irrational`s is that inequalities are correct:

In [9]:
Float64(pi) < pi < nextfloat(Float64(pi))

true

## &pi; via inline assembly instructions
*([Simon Byrne](https://github.com/simonbyrne))*

Julia provides a very low-level `llvmcall` interface, which allows the user to directly write [LLVM intermediate representation](http://llvm.org/docs/LangRef.html), including the use of inline assembly. The following snippet calls the `fldpi` instruction ("**f**loating point **l**oa**d** **pi**") which loads the constant &pi; onto the floating point register stack (this works only on x86 and x86_64 architectures)

In [10]:
function asm_pi()
    Base.llvmcall(
    """ %pi = call double asm "fldpi", "={st}"()
        ret double %pi""", 
    Float64, Tuple{})
end

asm_pi (generic function with 1 method)

In [11]:
asm_pi()

3.141592653589793

We can look at the actual resulting code that is generated:

In [12]:
@code_native asm_pi()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[10]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	fldpi
	fstpl	-8(%rbp)
	movsd	-8(%rbp), %xmm0         ## xmm0 = mem[0],zero
	popq	%rbp
	retq


If you're wondering what the rest of these instructions are doing:

1. the `pushq` and `movq` adds to the [call stack frame](https://en.wikipedia.org/wiki/Call_stack).
2. `fldpi` pushes &pi; to the x87 floating point register stack
  - x87 is the older legacy floating point instruction set dating back to the original [Intel 8087 coprocessor](https://en.wikipedia.org/wiki/Intel_8087).
3. `fstpl` and `movsd` moves the value to the SSE floating point register `xmm0`
  - Julia, like most modern software, uses the newer SSE instruction set for its floating point operations. This also allows us to take advantage of things like [SIMD operations](https://en.wikipedia.org/wiki/SIMD).
4. `popq` and `retq` pops the call stack frame.

## &pi; using a Taylor series expansions
*([Luis Benet](https://github.com/lbenet), Instituto de Ciencias Físicas, Universidad Nacional Autónoma de México (UNAM))*

This will demonstrate how to evaluate &pi; using various Taylor series expansions via the [TaylorSeries.jl](https://github.com/JuliaDiff/TaylorSeries.jl) package.

In [13]:
using TaylorSeries

### Madhava's formula

One of the standard trigonmetric identities is
$$ \tan\left( \frac{\pi}{6} \right) = \frac{1}{\sqrt{3}}. $$

Therefore, by taking the Taylor expansion of $6 \arctan(x)$ around 0 we may obtain the value of $\pi$, by evaluating it at $1/\sqrt{3}$, a value which is within the radius of convergence.

We obtain the Taylor series of order 37th, using `BigFloat`s:

In [14]:
series1 = 6atan( Taylor1(BigFloat, 37) )
convert(Taylor1{Rational{BigInt}},series1)

 6//1 t - 2//1 t³ + 6//5 t⁵ - 6//7 t⁷ + 2//3 t⁹ - 6//11 t¹¹ + 6//13 t¹³ - 2//5 t¹⁵ + 6//17 t¹⁷ - 6//19 t¹⁹ + 2//7 t²¹ - 6//23 t²³ + 6//25 t²⁵ - 2//9 t²⁷ + 6//29 t²⁹ - 6//31 t³¹ + 2//11 t³³ - 6//35 t³⁵ + 6//37 t³⁷ + 𝒪(t³⁸)

Note that the series above has only odd powers, so we will be using in this case 18 coefficients.

Evaluating that expression in $1/\sqrt{3}$ we get

In [15]:
pi_approx1 = evaluate(series1, 1/sqrt(big(3)))

3.141592653647826046431202390582141253830948237428790668441592864548346569098516

Then, the 37th order Taylor expansion yields a value which differs from $\pi$ in:

In [16]:
abs(pi - pi_approx1)

5.803280796855900730263836963377883805368484746664827224053016281231814650118929e-11

To obtain more accurate results, we may simply increase the order of the expansion:

In [17]:
series2 = 6atan( Taylor1(BigFloat,99) ) # 49 coefficients of the series
pi_approx2 = evaluate(series2, 1/sqrt(BigInt(3)))

3.141592653589793238462643347272152237127662423839333289949470742535834074912581

In [18]:
abs(pi - pi_approx2)

3.600735064706950697553577253102547384977198233137361734413175534929622111373249e-26

This formulation is one of the [*Madhava* or *Gregory–Leibniz series*](https://en.wikipedia.org/wiki/Madhava_series#Another_formula_for_the_circumference_of_a_circle):

\begin{equation}
\pi = 6 \sum_{n=0}^{\infty} (-1)^n \frac{(1/\sqrt{3})^{2n+1}}{2n+1}.
\end{equation}

### Machin's approach

Following the same idea, [John Machin](https://en.wikipedia.org/wiki/John_Machin#Formula) derived an algorithm which converges much faster, using the identity

\begin{equation}
\frac{\pi}{4} = 4 \arctan\left(\frac{1}{5}\right) - \arctan\left(\frac{1}{239}\right).
\end{equation}

Following what we did above, using again a 37th Taylor expansion:

In [19]:
ser = atan( Taylor1(BigFloat, 37) )
pi_approx3 = 4*( 4*evaluate(ser, 1/big(5)) - evaluate(ser, 1/big(239)) )

3.141592653589793238462643383496777424642594661632063407072684671069773618535135

In [20]:
abs(pi - pi_approx3)

2.17274540445425262256957586097740078761957212248936631045983596428448951876822e-28

## Finding guaranteed bounds on &pi;
*([David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/),  Department of Physics, Faculty of Sciences, National University of Mexico (UNAM)*) 

### Using standard floating-point arithmetic 

We will calculate *guaranteed* (i.e., *validated*, or mathematically rigorous) bounds on $\pi$ using just floating-point arithmetic. This requires "directed rounding", i.e. the ability to control in which direction floating-point operations are rounded.

This is based on the book [*Validated Numerics*](http://press.princeton.edu/titles/9488.html) (Princeton, 2011) by [Warwick Tucker](http://www2.math.uu.se/~warwick/CAPA/warwick/warwick.html).

Consider the infinite series

$$ S := \sum_{n=1}^\infty \frac{1}{n^2},$$

whose exact value is [known](http://en.wikipedia.org/wiki/Basel_problem) to be $S = \frac{\pi^2}{6}$.
Thus, if finding guaranteed bounds on $S$ will give guaranteed bounds on $\pi$. 

The idea is to split $S$ up into two parts, $S = S_N + T_N$, where
$ S_N := \sum_{n=1}^N \frac{1}{n^2}$ contains the first $N$ terms, 
and $T_N := S - S_N = \sum_{n=N+1}^\infty \frac{1}{n^2}$ contains the rest (an infinite number of terms).

We will evalute $S_N$ numerically, and use the following analytical bound for $T_N$:

$$\frac{1}{N+1} \le T_N \le \frac{1}{N}$$.

This is obtained by approximating the sum in $T_N$ using integrals from below and above:

$$\int_{x=N+1}^\infty \frac{1}{x^2} dx \le T_N \le \int_{x=N}^\infty \frac{1}{x^2} dx.$$

$S_N$ may be calculated easily by summing either forwards or backwards:

In [21]:
function forward_sum(N, T=Float64)
    total = zero(T)
    for i in 1:N
        total += one(T) / (i^2)
    end
    total
end

function reverse_sum(N, T=Float64)
    total = zero(T)
    for i in N:-1:1
        total += one(T) / (i^2)
    end
    total
end

reverse_sum (generic function with 2 methods)

To find *rigorous* bounds for $S_N$, we use "directed rounding", that is, we round downwards for the lower bound and  upwards for the upper bound:

In [22]:
N = 10^6

lowerbound_S_N = 
    setrounding(Float64, RoundDown) do
        forward_sum(N)
    end

upperbound_S_N = 
    setrounding(Float64, RoundUp) do
        forward_sum(N)
    end

(lowerbound_S_N, upperbound_S_N)

(1.6449330667377557,1.644933066959796)

We incorporate the respective bound on $T_N$ to obtain the bounds on $S$, and hence on $\pi$:

In [23]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = forward_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = forward_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π, lowerbound_S_N)

(3.1415926534833463,3.1415926536963346,1.6449330667377557)

In [24]:
upper_π - lower_π

2.1298829366855898e-10

We may check that the true value of $\pi$ is indeed contained in the interval:

In [25]:
lower_π < pi < upper_π

true

Summing in the opposite direction turns out to give a more accurate answer:

In [26]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = reverse_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = reverse_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π)

(3.1415926535893144,3.141592653590272)

In [27]:
upper_π - lower_π

9.57456336436735e-13

In [28]:
lower_π < pi < upper_π

true

In principle, we could attain arbitrarily good precision with higher-precision `BigFloat`s, but the result is hampered by the slow convergence of the series.

## Summing a series using interval arithmetic

We repeat the calculation using *interval arithmetic*, provided by the [ValidatedNumerics.jl](https://github.com/dpsanders/ValidatedNumerics.jl) package.

In [29]:
using ValidatedNumerics

In [30]:
setformat(:standard)  # abbreviated display of intervals

6

In [31]:
N = 10000
S = forward_sum(N, Interval)
S += 1/(N+1) .. 1/N  # interval bound on the remainder of the series
π_interval = √(6S)

[3.14159, 3.1416]

Here we used an abbreviated display for the interval. Let's see the whole thing:

In [32]:
setdisplay(:full)
π_interval

Interval(3.1415926488148807, 3.141592658365341)

It's diameter (width) is

In [33]:
diam(π_interval)

9.550460422502738e-9

Thus, the result is correct to approximately 8 decimals.

In this calculation, we used the fact that arithmetic operations of intervals with numbers automatically promote the numbers to an interval:

In [34]:
setdisplay(:full)  # full interval display
Interval(0) + 1/3^2

Interval(0.1111111111111111, 0.11111111111111112)

This is an interval containing the true real number $1/9$ (written `1//9` in Julia):

In [35]:
1//9 ∈ convert(Interval{Float64}, 1/3^2)

true

Finally, we can check that the true value of $\pi$ is indeed inside our interval:

In [36]:
pi ∈ π_interval

true

## Calculating an area

Although the calculation above is simple, the derivation of the series itself is not. In this section, we will use a more natural way to calculate $\pi$, namely that the area of a circle of radius $r$ is $A(r) = \pi r^2$. We will calculate the area of one quadrant of a circle of radius $r=2$, which is equal to $\pi$:

In [37]:
using Plots; gr();

In [38]:
f(x) = √(4 - x^2)

f (generic function with 1 method)

In [39]:
plot(f, 0, 2, aspect_ratio=:equal, fill=(0, :orange), alpha=0.2, label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
<polygon clip-path="url(#clip02)" points="
125.283,3.93701 159.69,5.83128 180.367,8.48391 197.849,11.6338 216.84,16.0233 234.347,20.9937 253.117,27.3498 274.864,36.1189 294.191,45.2676 311.138,54.4265 
 329.615,65.7341 348.202,78.6508 368.752,94.9811 388.305,112.86 408.636,134.403 425.75,155.483 436.197,170.031 446.644,186.187 456.465,203.221 466.285,222.62 
 474.152,240.5 482.019,261.433 485.952,273.57 489.885,287.362 493.819,303.645 497.752,324.397 498.963,332.468 500.173,342.064 501.384,354.602 501.384,384.952 
 500.173,384.952 498.963,384.952 497.752,384.952 493.819,384.952 489.885,384.952 485.952,384.952 482.019,384.952 474.152,384.952 466.285,384.952 456.465,384.952 
 446.644,384.952 436.197,384.952 425.75,384.952 408.636,384.952 388.305,384.952 368.752,384.952 348.202,384.952 329.615,384.952 311.138,384.952 294.191,384.952 
 274.864,384.952 253.117,384.952 234.347,384.952 216.84,384.952 197.849,384.952 180.367,384.952 159.69,384.952 125.283,384.952 
 " fill="#ffa500" fill-opacity="0.2"/>
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:0.2; fill:none" points="
 125.283,3.93701 159.69,5.83128 180.367,8.48391 197.849,11.6338 216.84,16.0233 234.347,20.9937 253.117,27.3498 274.864,36.1189 294.191,45.2676 311.138,54.4265 
 329.615,65.7341 348.202,78.6508 368.752,94.9811 388.305,112.86 408.636,134.403 425.75,155.483 436.197,170.031 446.644,186.187 456.465,203.221 466.285,222.62 
 474.152,240.5 482.019,261.433 485.952,273.57 489.885,287.362 493.819,303.645 497.752,324.397 498.963,332.468 500.173,342.064 501.384,354.602 
 "/>

The circle of radius $r=2$ is given by $x^2 + y^2 = 2^2 = 4$, so 

$$\pi = \frac{1}{4} A(2) = \int_{x=0}^2 y(x) \, dx = \int_{x=0}^2 \sqrt{4 - x^2}.$$

In calculus, we learn that we can approximate integrals using **Riemann sums**. Interval arithmetic allows us to make these Riemann sums **rigorous** in a very simple way, as follows.

We split up the $x$ axis into intervals, for example of equal width:

In [40]:
function make_intervals(N=10)
    xs = linspace(0, 2, N+1)
    return [xs[i]..xs[i+1] for i in 1:length(xs)-1]
end

intervals = make_intervals()

10-element Array{ValidatedNumerics.Interval{Float64},1}:
 Interval(0.0, 0.2)                               
 Interval(0.19999999999999998, 0.4)               
 Interval(0.39999999999999997, 0.6000000000000001)
 Interval(0.6, 0.8)                               
 Interval(0.7999999999999999, 1.0)                
 Interval(1.0, 1.2000000000000002)                
 Interval(1.2, 1.4000000000000001)                
 Interval(1.4, 1.6)                               
 Interval(1.5999999999999999, 1.8)                
 Interval(1.7999999999999998, 2.0)                

Given one of those intervals, we evaluate the function of interest:

In [41]:
II = intervals[1]

Interval(0.0, 0.2)

In [42]:
f(II)

Interval(1.9899748742132397, 2.0)

The result is an interval that is **guaranteed to contain** the true range of the function $f$ over that interval. So the lower and upper bounds of the intervals may be used as lower and upper bounds of the height of the box in a Riemann integral:

In [43]:
intervals = make_intervals(30)

p = plot(aspect_ratio=:equal)
for X in intervals
    Y = f(X)
    
    plot!(IntervalBox(X, Interval(0, Y.lo)), c=:blue, label="", alpha=0.1)
    plot!(IntervalBox(X, Interval(Y.lo, Y.hi)), c=:red, label="", alpha=0.1)
end

plot!(f, 0, 2)

p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 126.547,3.95518 160.952,5.84936 181.628,8.50187 199.11,11.6516 218.099,16.0409 235.606,21.0111 254.375,27.3669 276.121,36.1356 295.447,45.2838 312.393,54.4423 
 330.869,65.7493 349.455,78.6654 370.005,94.9949 389.557,112.873 409.886,134.415 427,155.494 437.446,170.042 447.893,186.196 457.713,203.23 467.533,222.627 
 475.399,240.507 483.266,261.439 487.199,273.576 491.132,287.367 495.065,303.649 498.998,324.4 500.209,332.47 501.42,342.067 502.63,354.603 
 "/>
 
 
 
 
 y61

Now we just sum up the areas:

In [44]:
N = 20
intervals = make_intervals(N)

width = 2/N
width * sum(√(4 - X^2) for X in intervals)

Interval(3.0284648797549782, 3.2284648797549846)

As we increase the number of sub-intervals, the approximation gets better and better:

In [45]:
setdisplay(:standard, sigfigs=5)

println("N \t area interval \t \t diameter")
for N in 50:50:1000
    intervals = make_intervals(N)
    area = (2/N) * sum(√(4 - X^2) for X in intervals)
            
    println("$N \t $area \t $(diam(area))")
end

N 	 area interval 	 	 diameter
50 	 [3.0982, 3.1783] 	 0.0800000000000165
100 	 [3.1204, 3.1605] 	 0.040000000000032454
150 	 [3.1276, 3.1543] 	 0.02666666666670814
200 	 [3.1311, 3.1512] 	 0.02000000000006308
250 	 [3.1332, 3.1493] 	 0.016000000000075065
300 	 [3.1346, 3.1481] 	 0.013333333333415354
350 	 [3.1356, 3.1472] 	 0.011428571428676815
400 	 [3.1364, 3.1465] 	 0.010000000000123688
450 	 [3.137, 3.146] 	 0.008888888889027502
500 	 [3.1374, 3.1455] 	 0.008000000000148333
550 	 [3.1378, 3.1452] 	 0.007272727272884527
600 	 [3.1381, 3.1449] 	 0.006666666666829357
650 	 [3.1384, 3.1446] 	 0.006153846154013376
700 	 [3.1386, 3.1444] 	 0.0057142857144931725
750 	 [3.1388, 3.1443] 	 0.005333333333562784
800 	 [3.139, 3.1441] 	 0.005000000000246363
850 	 [3.1391, 3.1439] 	 0.004705882353203794
900 	 [3.1393, 3.1438] 	 0.004444444444719142
950 	 [3.1394, 3.1437] 	 0.004210526316076102
1000 	 [3.1395, 3.1436] 	 0.004000000000294435
